In [168]:
!pip install -q kaggle unzip datasets
!pip install -q langchain_groq

#!/bin/bash
!kaggle datasets download adilmohammed/medical-data


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Dataset URL: https://www.kaggle.com/datasets/adilmohammed/medical-data
License(s): MIT
medical-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [169]:
# Necessary libraries 
import datasets
from datasets import load_dataset
from dotenv import load_dotenv
import os
from langchain_groq import ChatGroq

In [170]:
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=os.environ.get("GROQ_API_KEY"))
llm.invoke("Who's the current present of Rwanda ?").content

'The current President of Rwanda is Paul Kagame. He has been in office since 2000 and was re-elected for a third term in 2017 and then again for a fourth term in 2024 is not yet confirmed but he has been serving since 2000.'

In [180]:
context = "There are two populations: One is found in the Virunga volcanic " \
          "mountains of Central/East Africa, within three National Parks: Mgahinga, in southwest Uganda; Volcanoes, " \
          "in northwest Rwanda; and Virunga, in the eastern Democratic Republic of Congo (DRC). " \
          "The other population is found in Uganda's Bwindi Impenetrable National Park. " \
          "Some primatologists speculate the Bwindi population is a separate subspecies,[3] " \
          "though no description has been finalized. The latest population count, " \
          "released in 2019, revealed there to be approximately 1060 mountain " \
          "gorillas in the wild[4]"

In [159]:
# Langchain 
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

In [182]:
response_schemas = [
    ResponseSchema(name="Question", description="The generated question from the provided context"),
    ResponseSchema(name="Answer", description="The corresponding answer from the provided context")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions(only_json=True)

In [161]:
TEMPLATE = """ You are a helpful agent. Your task is to generate about 50 different or multiple meaningful questions with their corresponding answers using the following provided "{context}" \n\n
"You MUST obey the following criteria:\n"
"- No pre-amble.\n"
"- Restrict the question to the context information provided.\n"
"- Do NOT create a question that cannot be answered from the context.\n"
"- Phrase the question so that it does NOT refer to specific context.\n"
"- For instance, do NOT use phrases like 'given the provided context' or 'in this work' in the question or 'according to the text' in the answer becuse if the question is asked elsewher it would not be provided specific context. Replace these terms with specific details.\n"
"- Please do NOT repeat the provided context.\n"
"- Please follow JSON recommended format below.\n"
"- Please ensure that the ouput is a valid JSON object.\n"
"{format_instructions}"""

prompt = ChatPromptTemplate.from_template(template=TEMPLATE)


In [162]:
messages = prompt.format_messages(context=context, format_instructions=format_instructions)
response = llm.invoke(messages)
response

AIMessage(content='```json\n[\n  {\n    "Question": "Where are mountain gorillas primarily found?",\n    "Answer": "Central/East Africa"\n  },\n  {\n    "Question": "What are the names of the National Parks where mountain gorillas are found?",\n    "Answer": "Mgahinga, Volcanoes, and Virunga"\n  },\n  {\n    "Question": "In which countries are the Virunga volcanic mountains located?",\n    "Answer": "Uganda, Rwanda, and the Democratic Republic of Congo"\n  },\n  {\n    "Question": "What is the name of the National Park in Uganda where the other population of mountain gorillas is found?",\n    "Answer": "Bwindi Impenetrable National Park"\n  },\n  {\n    "Question": "Do some primatologists consider the Bwindi population to be a separate subspecies?",\n    "Answer": "Yes"\n  },\n  {\n    "Question": "Has a description of the Bwindi population as a separate subspecies been finalized?",\n    "Answer": "No"\n  },\n  {\n    "Question": "What was the approximate number of mountain gorillas in

In [163]:
import json

# Extract the content from the AIMessage object
response_content = response.content

# Remove the Markdown code block markers (```json and ```)
if response_content.startswith('```json') and response_content.endswith('```'):
    response_content = response_content[7:-3].strip()  # Remove ```json and ```

# Parse the JSON string
try:
    data = json.loads(response_content)
    
    # Iterate through the list and print each question-answer pair
    for item in data:
        question = item.get("Question", "No Question Found")
        answer = item.get("Answer", "No Answer Found")
        print(f"Q: {question}")
        print(f"A: {answer}")
        print()
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)

Q: Where are mountain gorillas primarily found?
A: Central/East Africa

Q: What are the names of the National Parks where mountain gorillas are found?
A: Mgahinga, Volcanoes, and Virunga

Q: In which countries are the Virunga volcanic mountains located?
A: Uganda, Rwanda, and the Democratic Republic of Congo

Q: What is the name of the National Park in Uganda where the other population of mountain gorillas is found?
A: Bwindi Impenetrable National Park

Q: Do some primatologists consider the Bwindi population to be a separate subspecies?
A: Yes

Q: Has a description of the Bwindi population as a separate subspecies been finalized?
A: No

Q: What was the approximate number of mountain gorillas in the wild in 2019?
A: 1060

Q: In which year was the latest population count of mountain gorillas released?
A: 2019

Q: How many National Parks are home to mountain gorillas?
A: Four

Q: What type of mountains are the mountain gorillas found in?
A: Volcanic mountains

Q: Are mountain gorillas fo

In [164]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "What is the capital city of Rwanda ?",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

The capital city of Rwanda is Kigali.


In [165]:
ds = load_dataset("lavita/medical-qa-datasets", "chatdoctor-icliniq")

In [166]:
ds['test'][0]

{'input': 'Hello doctor,I had mumps five months ago and after that, I started to have an infection in my left testes. It was swollen and now it has shrunk to almost half the size of the other one. As I am sexually active, I feel a pain in each of the vas deferens after sex. If I do not have sex for days, they become sensitive. I was treated with Ceftum 500 mg, the first time I had an infection. Now my question is, is there any chance that the infection is still in my body? And, do I need to get examined for it? For the time being, please suggest some precautionary antibiotics for my relief.',
 'answer_icliniq': 'Hello, Welcome to Chat Doctor forum. I can understand your concern. You had mumps and this is a viral infection known to cause an inflammation of the testis in some cases. Take care. For more information consult a sexologist online',
 'answer_chatgpt': "It is possible that you still have an infection in your testes, especially if you have been experiencing pain and sensitivity 

In [167]:
ds['test'][1]

{'input': 'Hello doctor,I am trying to conceive but my husband and I did cocaine a week ago. How long should my husband and I wait to safely continue to try to get pregnant? How long until it is out of our system? How long does cocaine stay in sperm? Thanks in advance.',
 'answer_icliniq': 'Hello, Wellcome to ChatDoctor. There are few researches/studies on cocaine use by males and its effect on pregnancy. Few suggest that cocaine by itself has limited effects as most of the time it is taken along with other  ChatDoctor.  So, most of the people take not just cocaine but a combination of',
 'answer_chatgpt': 'Hello! Thank you for reaching out.\\n\\nCocaine use can have negative effects on fertility and pregnancy. Both you and your husband should wait until the drug has completely cleared from your systems before trying to conceive. \\n\\nCocaine and its metabolites can be detected in urine for up to 72 hours after use, but it can take longer for the drug to clear from the system complete